# Setup or Reset

## Warning!!!!
### Running this code will delete an existing database as specified in the state dictionary below.

We will start with a wrapper around Snowpark to simplify connections and the ability to choose different compute instances.  We need a way to save state throughout the project too.  
  
Update the \<USERNAME>, \<ACCOUNTNAME> and \<DOMAIN> in the state dictionary below.

In [ ]:
state_dict = {
    "connection_parameters": {"user": "<USERNAME>",  
                              "database": "CITIBIKEML_<USERNAME>",  
                              "schema": "DEMO",  
                              "password": "",
                              "account": "<ACCOUNTNAME>.<DOMAIN>",
                              "role": "PUBLIC",  
    },
    "compute_parameters" : {
              "default_warehouse": "XSMALL_WH",  
              "task_warehouse": "XSMALL_WH",  
              "load_warehouse": "LARGE_WH",  
              "fe_warehouse": "XXLARGE_WH",  
              "train_warehouse": "XXLARGE_WH"  
    },
    "download_base_url":"https://s3.amazonaws.com/tripdata/",
    "load_table_name":"RAW_",
    "trips_table_name":"TRIPS",
    "load_stage_name":"LOAD_STAGE",
    "model_stage_name":"MODEL_STAGE",
    "model_id": "",
    "key_pair_name":"", 
    "public_key":""
}

In [ ]:
import json
with open('./include/state.json', 'w') as sdf:
    json.dump(state_dict, sdf)

We will connect with username and password.

In [ ]:
import snowflake.snowpark as snp
import os, json, uuid
import getpass

with open('./include/state.json') as sdf:
    state_dict = json.load(sdf)    
state_dict['model_id'] = str(uuid.uuid1()).replace('-', '_')
state_dict['connection_parameters']['password'] = getpass.getpass()

session = snp.Session.builder.configs(state_dict["connection_parameters"]).create()

If you do not currently have these users and roles created you can use something like the following to create them.

In [ ]:
# session.use_role('securityadmin')

# initial_password='snow2022'
# new_usernames=['john1', 'john2', 'john3']
# project_role='PUBLIC'

# for username in new_usernames:
#     session.sql("CREATE USER IF NOT EXISTS "+username+\
#                 " PASSWORD = '"+initial_password+\
#                 "' LOGIN_NAME = '"+username+\
#                 "' DEFAULT_ROLE = '"+project_role+\
#                 "' MUST_CHANGE_PASSWORD = TRUE")\
#             .collect()
#     session.sql("GRANT ROLE "+project_role+" TO USER "+username).collect()
#     #session.sql("DROP USER "+username).collect()

# session.use_role('sysadmin')
# session.sql("GRANT CREATE DATABASE ON ACCOUNT TO ROLE "+project_role).collect()

# session.use_role(state_dict['connection_parameters']['role'])

Likewise for compute warehouses the following code example can create them automatically.

In [ ]:
# session.use_role('accountadmin')
# project_role='PUBLIC'

# for wh in state_dict['compute_parameters'].values():
#     session.sql("CREATE WAREHOUSE IF NOT EXISTS "+wh+\
#             " WITH WAREHOUSE_SIZE = '"+wh.split('_')[0]+\
#             "' WAREHOUSE_TYPE = 'STANDARD' AUTO_SUSPEND = 300 AUTO_RESUME = TRUE initially_suspended = true;")\
#         .collect()
#     session.sql("GRANT USAGE ON WAREHOUSE "+wh+" TO ROLE "+project_role).collect() 
    
# session.use_role(state_dict['connection_parameters']['role'])

Quick test of changing compute types

In [ ]:
session.get_current_warehouse()

In [ ]:
for wh in state_dict['compute_parameters'].keys():
    session.use_warehouse(state_dict['compute_parameters'][wh])

In [ ]:
session.get_current_warehouse()

In [ ]:
session.close()

Save the updated state dictionary for project team use.

In [ ]:
import json
with open('./include/state.json', 'w') as sdf:
    json.dump(state_dict, sdf)

 We add a python function to simplify starting a session.  We will add a new dictionary of compute resources available to our DE and ML Ops pipeline. 

In [ ]:
%%writefile dags/snowpark_connection.py

def snowpark_connect(state_file='./include/state.json'):
    import snowflake.snowpark as snp
    import os, json, uuid
    
    with open(state_file) as sdf:
        state_dict = json.load(sdf)    
    state_dict["model_id"] = str(uuid.uuid1()).replace('-', '_')
    
    session=None
    session = snp.Session.builder.configs(state_dict["connection_parameters"]).create()
    session.use_warehouse(state_dict['compute_parameters']['default_warehouse'])
    return session, state_dict

Test the function that users will use.

In [ ]:
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect()

Create the database and schema for this project.

In [ ]:
_ = session.sql('CREATE OR REPLACE DATABASE '+state_dict['connection_parameters']['database']).collect()
_ = session.sql('CREATE SCHEMA '+state_dict['connection_parameters']['schema']).collect() 

In [ ]:
session.close()